In [198]:
#In this example, you'll be creating a Python script to visualize the weather of 500+ cities across the world of varying distance from the equator. To accomplish this, you'll be utilizing a [simple Python library](https://pypi.python.org/pypi/citipy), the [OpenWeatherMap API](https://openweathermap.org/api), and a little common sense to create a representative model of weather across world cities.

1st Objective - Scatter Plots

In [199]:
#Your first objective is to build a series of scatter plots to showcase the following relationships:

#* Temperature (F) vs. Latitude
#* Humidity (%) vs. Latitude
#* Cloudiness (%) vs. Latitude
#* Wind Speed (mph) vs. Latitude

#After each plot add a sentence or two explaining what the code is and analyzing.

In [200]:
#Import dependencies
from citipy import citipy
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from config import api_key
import random
import decimal
import requests
from time import sleep

In [201]:
###########create random list of latitudes and longitude - REPLACE 5 WITH 500 LATER ON
latitudes = []
longitudes = []

for i in range(0,5):
    x = float(decimal.Decimal(random.randint(-18000,18000))/100)
    latitudes.append(x)

lat_df = pd.DataFrame(latitudes, columns = ["Latitude"])

for y in range(0,5):
    y = float(decimal.Decimal(random.randint(-18000,18000))/100)
    longitudes.append(y)

long_df = pd.DataFrame(longitudes, columns=["Longitude"])

In [202]:
#combine Latitude and Longitude lists to get coordinates
coord_df = pd.concat([lat_df,long_df], axis=1)
#coordinates_df = coord_df
coord_df.head()

,Latitude,Longitude
0,-142.17,-79.39
1,139.72,17.96
2,-94.73,87.75
3,165.70,-151.81
4,109.57,43.93


In [228]:
#find city closest to the coordinates created
#loop through the latitude and longitude for each set of coords to pull city from citipy
cities = []
cc = []
for index, row in coord_df.iterrows():
    lat = row["Latitude"]
    long = row["Longitude"]

    city = citipy.nearest_city(lat,long)
    cities.append(city.city_name)

cities_df = pd.DataFrame(cities, columns = ["City Name"])

cities_df = pd.concat([cities_df,coord_df], axis =1)

cities_df.head()

,City Name,Latitude,Longitude
0,ushuaia,-142.17,-79.39
1,longyearbyen,139.72,17.96
2,albany,-94.73,87.75
3,barrow,165.70,-151.81
4,belushya guba,109.57,43.93


In [204]:
#set-up URLs
units = "imperial"
base_url = f"http://api.openweathermap.org/data/2.5/weather?units={units}&appid={api_key}&q="


In [229]:
#pull weather data from open weather
temp = []
humid = []
clouds = []
windspeed = []
city_lats = []
city_long = []

for city in cities_df["City Name"]:
    response = requests.get(base_url+city).json()
    pprint(response)
    #append the list

    try:
        temp.append(response["main"]["temp"])
        humid.append(response["main"]["humidity"])
        clouds.append(response["clouds"]["all"])
        windspeed.append(response["wind"]["speed"])
        city_lats.append(response["coord"]["lat"])
        city_long.append(response["coord"]["lon"])

    except:
        print(f"{city} not found!")
        pass
    #add pause so loop doesn't time out
    sleep(0.5)


{'base': 'model',
 'clouds': {'all': 100},
 'cod': 200,
 'coord': {'lat': -54.8, 'lon': -68.3},
 'dt': 1579984818,
 'id': 3833367,
 'main': {'feels_like': 35.62,
          'grnd_level': 943,
          'humidity': 57,
          'pressure': 991,
          'sea_level': 991,
          'temp': 47.46,
          'temp_max': 47.46,
          'temp_min': 47.46},
 'name': 'Ushuaia',
 'rain': {'3h': 0.44},
 'sys': {'country': 'AR', 'sunrise': 1579941661, 'sunset': 1579999771},
 'timezone': -10800,
 'weather': [{'description': 'light rain',
              'icon': '10d',
              'id': 500,
              'main': 'Rain'}],
 'wind': {'deg': 265, 'speed': 14.94}}
{'base': 'model',
 'clouds': {'all': 0},
 'cod': 200,
 'coord': {'lat': 78.22, 'lon': 15.64},
 'dt': 1579985354,
 'id': 2729907,
 'main': {'feels_like': -19.97,
          'grnd_level': 975,
          'humidity': 58,
          'pressure': 1018,
          'sea_level': 1018,
          'temp': -11.52,
          'temp_max': -11.52,
          '

In [225]:
#build temp (f) vs latitude scatterplot
plt.scatter(temp,coord_df["Latitude"], marker = "o")
plt.title("Temperature (F) vs. Latitude")
plt.xlabel("Temperature (F)")
plt.ylabel("Latitude")
plt.grid(True)

ValueError: x and y must be the same size

In [0]:
#In the comparison of Temperature vs Latitude, the code is looking at where the city sits by Latitude on the globe and then looking at it's current temperature. The scatterplot looks at the correlation between distance from the equator to the current temperature.

In [0]:
#* Humidity (%) vs. Latitude
plt.scatter(humid,coord_df["Latitude"], marker = "o")
plt.title("Humidity (%) vs. Latitude")
plt.xlabel("Humidity (%)")
plt.ylabel("Latitude")
plt.grid(True)


In [0]:
#In the comparison of Humidty % vs Latitude, the code is looking at where the city sits by Latitude on the globe and then looking at it's current humidity %. The scatterplot is showing how distance from the equator might affect the level of humidity in a certain city.

In [0]:
#* Cloudiness (%) vs. Latitude
plt.scatter(clouds,coord_df["Latitude"], marker = "o")
plt.title("Cloudiness (%) vs. Latitude")
plt.xlabel("Cloudiness (%)")
plt.ylabel("Latitude")
plt.grid(True)

In [0]:
#In the comparison of Cloudiness % vs Latitude, the code is looking at where the city sits by Latitude on the globe and then pulling it's current Cloud coverage %. The scatterplot is showing how distance from the equator might affect the cloudiness of a certain city.

In [0]:
#* Wind Speed (mph) vs. Latitude
plt.scatter(windspeed,coord_df["Latitude"], marker = "o")
plt.title("Windspeed (mph) vs. Latitude")
plt.xlabel("Windspeed (mph)")
plt.ylabel("Latitude")
plt.grid(True)

In [0]:
#In the comparison of Windspeed (mph) vs Latitude, the code is looking at where the city sits by Latitude on the globe and then pulling it's current wind speed. The scatterplot is showing how distance from the equator might affect windiness of a certain city.

2nd Objective - Linear Regression

In [0]:
#Your next objective is to run linear regression on each relationship, only this time separating them into Northern Hemisphere (greater than or equal to 0 degrees latitude) and Southern Hemisphere (less than 0 degrees latitude):

#* Northern Hemisphere - Temperature (F) vs. Latitude
#* Southern Hemisphere - Temperature (F) vs. Latitude
#* Northern Hemisphere - Humidity (%) vs. Latitude
#* Southern Hemisphere - Humidity (%) vs. Latitude
#* Northern Hemisphere - Cloudiness (%) vs. Latitude
#* Southern Hemisphere - Cloudiness (%) vs. Latitude
#* Northern Hemisphere - Wind Speed (mph) vs. Latitude
#* Southern Hemisphere - Wind Speed (mph) vs. Latitude

#After each pair of plots explain what the linear regression is modelling such as any relationships you notice and any other analysis you may have.

In [0]:
coordinates_df = pd.concat([coord_df, cities_df], axis = 1)
coordinates_df.head()

In [0]:
#determine northern and southern hemisphere cities
n_cities = []
s_cities = []

for city in cities_df["Latitude"]:
    north = if city > 0 then
        n_cities.append(city)

In [0]:
#Linear regression on Northern Hemisphere - Temperature (F) vs. Latitude
#

In [0]:
#Your final notebook must:

#* Randomly select **at least** 500 unique (non-repeat) cities based on latitude and longitude.
#* Perform a weather check on each of the cities using a series of successive API calls.
#* Include a print log of each city as it's being processed with the city number and city name.
#* Save a CSV of all retrieved data and a PNG image for each scatter plot.
